In [1]:
import pandas as pd 
import numpy as np 
df1=pd.read_csv('./lingo_ex_movie.csv')
df2=pd.read_csv('./lingo_ex_rating.csv')
df1.head(3)

,index,belongs_to_collection,genres,id,production_companies,release_date,title,vote_average,vote_count
0,0,Toy Story Collection,"['Animation', 'Comedy', 'Family']",862,Pixar Animation Studios,1995-10-30,Toy Story,7.7,5415.0
1,1,NaN,"['Adventure', 'Fantasy', 'Family']",8844,TriStar Pictures,1995-12-15,Jumanji,6.9,2413.0
2,2,Grumpy Old Men Collection,"['Romance', 'Comedy']",15602,Warner Bros.,1995-12-22,Grumpier Old Men,6.5,92.0


In [2]:
df2.head(3)

,userId,movieId,rating,date
0,1,110,1.0,2015-03-09
1,1,147,4.5,2015-03-09
2,1,858,5.0,2015-03-09


In [11]:
test = df2.groupby(['date']).size().reset_index(name='counts')
test.head(3)

,date,counts
0,1995-01-09,4
1,1996-01-29,62
2,1996-01-30,4


In [16]:
test = test.sort_values(by= ['counts'], ascending=False)
test.head(15)

,date,counts
1682,2000-11-20,97319
3264,2005-03-22,79428
1337,1999-12-11,68751
4579,2008-10-29,57010
1683,2000-11-21,56050
3265,2005-03-23,54897
1339,1999-12-13,52914
1338,1999-12-12,49795
1681,2000-11-19,43972
1340,1999-12-14,39686
